<a href="https://colab.research.google.com/github/Ar11-kgp/Risk_Parity_Portfolio_Optimization/blob/main/Risk_Parity_Portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Implementation of the Risk Parity Portfolio

Author:
*   Name: Arya Rajendra
*   Year of study: 4th Year
*   College: IIT Kharagpur
*   Under the guidence of: Kshitij Anand

In [ ]:
import yfinance as yf
from scipy.optimize import minimize # Import the minimize function

##Downloading the 5 months data

In [ ]:
import pandas as pd
import numpy as np

# List of six diversified stocks (e.g., tech, healthcare, finance, consumer goods, industrials, energy)
stocks = ['AAPL', 'JNJ', 'JPM', 'PG', 'BA', 'XOM']

# Download historical data for the past 5 months
data = yf.download(stocks, start='2024-02-01', end='2024-07-01')['Adj Close']

# Calculate daily returns
returns = data.pct_change().dropna()
returns

[*********************100%%**********************]  6 of 6 completed


Ticker,AAPL,BA,JNJ,JPM,PG,XOM
Date,,,,,,
2024-02-02,-0.005405,-0.002049,-0.011051,0.005756,-0.006848,-0.004102
2024-02-05,0.009847,-0.013134,-0.005172,-0.001316,0.000759,-0.004119
2024-02-06,0.008632,0.009437,0.014506,0.003438,0.004741,0.006893
2024-02-07,0.000581,0.016013,-0.000506,0.001885,0.001006,-0.000293
2024-02-08,-0.005755,-0.012741,-0.010001,-0.003591,-0.003017,0.017120
...,...,...,...,...,...,...
2024-06-24,0.003133,0.014386,0.002487,0.013143,0.001129,0.029704
2024-06-25,0.004468,-0.022334,-0.012943,-0.004073,-0.009498,0.002806
2024-06-26,0.019993,0.019417,-0.002514,-0.003231,0.003596,0.000350


In [ ]:
# The variance-covariance matrix
C = returns.cov()
print("Covariance Matrix:")
print(C)

Covariance Matrix:
Ticker          AAPL        BA           JNJ       JPM        PG       XOM
Ticker                                                                    
AAPL    2.456415e-04  0.000037  4.926328e-07 -0.000031  0.000002 -0.000007
BA      3.664455e-05  0.000328  4.797165e-05  0.000043  0.000029  0.000027
JNJ     4.926328e-07  0.000048  9.497306e-05  0.000039  0.000027  0.000009
JPM    -3.125948e-05  0.000043  3.926331e-05  0.000155  0.000019  0.000043
PG      2.409910e-06  0.000029  2.729974e-05  0.000019  0.000047  0.000012
XOM    -6.613172e-06  0.000027  8.981707e-06  0.000043  0.000012  0.000124


###METHOD-1 IMPLEMTATION

In [ ]:
w = np.ones(6)/6   #initial guess for weight matrix
C = np.array(C)

def obj_function(w, C):
    P = C @ w
    MCR = P/np.sqrt(w.T @ C @ w)
    f = 0.0 # obj function value initialization

    for i in range(len(w)):
        for j in range(len(w)):
            f += (w[i] * MCR[i] - w[j] * MCR[j]) ** 2 # summation of sq diff of all possible risk contri comb.
    return f

def objective(w):
    return obj_function(w, C)

# Constraints
constraints = (
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # Equality constraint: sum(w) = 1
    {'type': 'ineq', 'fun': lambda w: w}             # Inequality constraint: w >= 0
)

# Bounds
bounds = [(0, 1) for _ in range(C.shape[1])]

# Optimize
result = minimize(objective, w, method='SLSQP', constraints=constraints, bounds=bounds, options={'disp': True})

# Optimal weights
optimal_weights = result.x

print("Optimal weights:", optimal_weights)
print("Objective function value at optimal weights:", result.fun)
print("Success:", result.success)
print("Message:", result.message)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.7285584312807677e-05
            Iterations: 1
            Function evaluations: 7
            Gradient evaluations: 1
Optimal weights: [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
Objective function value at optimal weights: 1.7285584312807677e-05
Success: True
Message: Optimization terminated successfully


###METHOD-2 IMPLEMNTATION

In [ ]:
w = np.ones(6)/6   #initial guess for weight matrix
C = np.array(C)

def obj_function(w, C):
    return np.sqrt(w.T @ C @ w)


def objective(w):
    return obj_function(w, C)

# Trying for different values of c
constants = [4, 3, 2, 1]
best_portfolio = {
    "weights": None,
    "return": None,
    "variance": float('inf'),  # Initialize with infinity
    "constant": None
}

# Calculating expected returns and covariance matrix
expected_returns = returns.mean()
cov_matrix = returns.cov()

for c in constants:
    print(f"Testing with constant: {c}")
    constraints = (
        {'type': 'ineq', 'fun': lambda w: np.sum(np.log(w)) + c},  # Inequality constraint
        {'type': 'ineq', 'fun': lambda w: w}  # Non-negativity constraint
    )

    # Bounds
    bounds = [(0, 1) for _ in range(cov_matrix.shape[0])]

    # Optimization
    result = minimize(objective, w, method='SLSQP', constraints=constraints, bounds=bounds, options={'disp': False})

    if result.success:
        # Optimal weights
        optimal_weights = result.x
        normalized_weights = optimal_weights / np.sum(optimal_weights)

        # Calculating portfolio expected return and variance
        portfolio_return = np.dot(normalized_weights, expected_returns)
        portfolio_variance = np.dot(normalized_weights.T, np.dot(cov_matrix, normalized_weights))

        # Checking if this portfolio has the lowest variance
        if portfolio_variance < best_portfolio["variance"]:
            best_portfolio["weights"] = normalized_weights
            best_portfolio["return"] = portfolio_return
            best_portfolio["variance"] = portfolio_variance
            best_portfolio["constant"] = c

        # Printing results for each constant
        print(f"Constant {c}:")
        print("Normalized weights:", normalized_weights)
        print("Expected Portfolio Return:", portfolio_return)
        print("Portfolio Variance:", portfolio_variance)
        print("Success:", result.success)
        print("Message:", result.message)
        print()

Testing with constant: 4
Constant 4:
Normalized weights: [0.16053918 0.0953542  0.17250124 0.1476746  0.24753082 0.17639996]
Expected Portfolio Return: 0.0005848313595151965
Portfolio Variance: 3.6359028433233126e-05
Success: True
Message: Optimization terminated successfully

Testing with constant: 3
Constant 3:
Normalized weights: [0.16055013 0.09535677 0.17252416 0.14766989 0.24749937 0.17639968]
Expected Portfolio Return: 0.0005848075116653912
Portfolio Variance: 3.635985120951649e-05
Success: True
Message: Optimization terminated successfully

Testing with constant: 2
Constant 2:
Normalized weights: [0.16368544 0.09811225 0.17902614 0.15220791 0.22572829 0.18123997]
Expected Portfolio Return: 0.000586372329861719
Portfolio Variance: 3.706972082108579e-05
Success: True
Message: Optimization terminated successfully

Testing with constant: 1
Constant 1:
Normalized weights: [0.1699575  0.10234265 0.18764302 0.15831083 0.19267741 0.18906858]
Expected Portfolio Return: 0.000590804944293

##Best portfolio out of all the values of c

In [ ]:
# The best portfolio is the one with lowest variance out of all
print("Best Portfolio:")
print("Constant:", best_portfolio["constant"])
print("Weights:", best_portfolio["weights"])
print("Expected Return:", best_portfolio["return"])
print("Variance:", best_portfolio["variance"])

Best Portfolio:
Constant: 4
Weights: [0.16053918 0.0953542  0.17250124 0.1476746  0.24753082 0.17639996]
Expected Return: 0.0005848313595151965
Variance: 3.6359028433233126e-05


## Fetching Market Capitalization Data

In [ ]:
# Fetching market capitalization data for the selected stocks
market_caps = {}

for stock in stocks:
    ticker = yf.Ticker(stock)
    market_caps[stock] = ticker.info['marketCap']

market_caps = np.array([market_caps[stock] for stock in stocks])

print('Market Capitalizations (in USD):', market_caps)


Market Capitalizations (in USD): [3499701567488  352843366400  591793750016  391500038144  113727971328
  503904534528]


##Comparison with Marekt Cap Weighted Portfolio

In [ ]:
# Market Cap Weighted Portfolio
market_cap_weights = market_caps / market_caps.sum()
market_cap_portfolio_variance = np.dot(market_cap_weights.T, np.dot(cov_matrix, market_cap_weights))

In [ ]:
# Final Comparison
print('Optimized Portfolio Variance:', best_portfolio["variance"])
print('Market Cap Weighted Portfolio Variance:', market_cap_portfolio_variance)

Optimized Portfolio Variance: 3.6359028433233126e-05
Market Cap Weighted Portfolio Variance: 0.00010809570207153585


##Conclusions:



*   The optimized portfolio variance (3.6359028433233126e-05) is significantly lower than the market cap weighted portfolio variance (0.00010809570207153585).
This indicates that the optimization method successfully minimized the portfolio variance, leading to a more efficient portfolio in terms of risk.

*   The significantly lower portfolio variance achieved through optimization demonstrates the effectiveness of using mathematical optimization techniques to construct a low-risk portfolio. In contrast, the market cap weighted portfolio, while simpler to implement, does not account for the covariance structure between assets and thus results in higher risk. This analysis underscores the value of optimization in portfolio management, particularly for investors prioritizing risk minimization.

